In [1]:
from __future__ import print_function
import mxnet as mx
from mxnet import nd, autograd
from mxnet import gluon
import numpy as np
mx.random.seed(1)

In [2]:
ctx = mx.cpu()

In [3]:
batch_size = 64
num_inputs = 784
num_outputs = 10
def transform(data, label):
    return nd.transpose(data.astype(np.float32), (2,0,1))/255, label.astype(np.float32)

In [4]:
train_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=True, transform=transform),
                                      batch_size, shuffle=True)
test_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=False, transform=transform),
                                     batch_size, shuffle=False)

In [6]:
num_fc = 512
net = gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Conv2D(channels=20, kernel_size=5, activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
    net.add(gluon.nn.Conv2D(channels=50, kernel_size=5, activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
    # The Flatten layer collapses all axis, except the first one, into one axis.
    net.add(gluon.nn.Flatten())
    net.add(gluon.nn.Dense(num_fc, activation="relu"))
    net.add(gluon.nn.Dense(num_outputs))

In [7]:
net.collect_params().initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)

In [8]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

In [9]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': .1})

In [10]:
def evaluate_accuracy(data_iterator, net):
    acc = mx.metric.Accuracy()
    for i, (data, label) in enumerate(data_iterator):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=label)
    return acc.get()[1]

In [11]:
epochs = 10
smoothing_constant = .01

for e in range(epochs):
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(data.shape[0])

        ##########################
        #  Keep a moving average of the losses
        ##########################
        curr_loss = nd.mean(loss).asscalar()
        moving_loss = (curr_loss if ((i == 0) and (e == 0))
                       else (1 - smoothing_constant) * moving_loss + (smoothing_constant) * curr_loss)

    test_accuracy = evaluate_accuracy(test_data, net)
    train_accuracy = evaluate_accuracy(train_data, net)
    print("Epoch %s. Loss: %s, Train_acc %s, Test_acc %s" % (e, moving_loss, train_accuracy, test_accuracy))

Epoch 0. Loss: 0.0800091518561, Train_acc 0.97915, Test_acc 0.9825
Epoch 1. Loss: 0.0549273134731, Train_acc 0.985383333333, Test_acc 0.9859
Epoch 2. Loss: 0.0339950045533, Train_acc 0.991166666667, Test_acc 0.9893
Epoch 3. Loss: 0.0309011697333, Train_acc 0.99375, Test_acc 0.9899
Epoch 4. Loss: 0.0231434528234, Train_acc 0.996016666667, Test_acc 0.9912
Epoch 5. Loss: 0.0153818447652, Train_acc 0.9967, Test_acc 0.9918
Epoch 6. Loss: 0.0175201073106, Train_acc 0.994566666667, Test_acc 0.9883
Epoch 7. Loss: 0.0104700706029, Train_acc 0.991366666667, Test_acc 0.9844
Epoch 8. Loss: 0.0114457445613, Train_acc 0.998483333333, Test_acc 0.9922
Epoch 9. Loss: 0.00692503644025, Train_acc 0.9989, Test_acc 0.9923
